# **Reporte del Modelo Baseline**
---

In [2]:
# Librerias
!pip install keras-tuner
import os
import requests
import logging
import keras_tuner
import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import layers
from sklearn.metrics import accuracy_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 10.2 MB/s eta 0:00:00


In [3]:
# Funcion para crear el objeto de creación de logs
def create_logger():
  logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(levenname)s - %(message)s')
  logger = logging.getLogger('Logger')
  logger.info('Logger creado')
  return logger

In [4]:
# Función de consumo a la base de datos de Firebase para cargar el DataSet
def download_firebase(url, logger):
  logger.info("Extrayendo el archivo desde Firebase")
  df = None
  try:
    df = pd.read_csv(url)
    logger.info("Archivo cargado")
  except requests.exceptions.RequestException as e:
    logger.info(f"Error al descargar el archivo CSV: {e}")
  except pd.errors.EmptyDataError:
    logger.info("El archivo CSV está vacío.")
  except Exception as e:
    logger.info(f"Ocurrió un error inesperado: {e}")
  return df

In [5]:
# Cargar DataSet
url = 'https://firebasestorage.googleapis.com/v0/b/personalwp-8822c.appspot.com/o/diabetes_prediction_dataset.csv?alt=media&token=4d70d154-c3d0-4fa0-a3aa-9b9972dd3b95'
logger = create_logger()
df = download_firebase(url, logger)

In [6]:
# Eliminación de valores atípicos y duplicados
seventy_fifth = df['bmi'].quantile(0.75)
twenty_fifth = df['bmi'].quantile(0.25)
iqr = seventy_fifth - twenty_fifth
upper = seventy_fifth + (10 * iqr)
outliers_bmi_upper = df[(df['bmi'] > upper)]
df = pd.merge(df, outliers_bmi_upper, indicator = True, how = 'outer').query('_merge == "left_only"').drop('_merge', axis = 1)
df = df.drop_duplicates(keep = "first")

In [7]:
# Variables Categóricas a Numéricas
df['gender'] = pd.factorize(df['gender'])[0]
df['smoking_history'] = pd.factorize(df['smoking_history'])[0]

## **Implementación del Modelo**

### **Partición de Datos**

In [8]:
# Separación de la 'Data' (Características)
X = df.drop(columns = 'diabetes')
X.shape

(96143, 8)

In [9]:
# Separación del 'Target' (Variable objetivo)
y = df['diabetes']
y.shape

(96143,)

In [10]:
# Partición de los datos: 70% para entrenamiento, 10% para validación, 20% para prueba y estratificación en las etiquetas (y)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size = 0.125, random_state = 42, stratify = y_temp)

In [11]:
# Validación de la partición de los datos
print(f'Número de muestras en entrenamiento: {X_train.shape[0]}')
print(f'Número de muestras en validación: {X_val.shape[0]}')
print(f'Número de muestras en prueba: {X_test.shape[0]}')
print(f'Número de características: {X_train.shape[1]}')

Número de muestras en entrenamiento: 67299
Número de muestras en validación: 9615
Número de muestras en prueba: 19229
Número de características: 8


In [12]:
# Distribución de la variable objetivo en los conjuntos de entrenamiento y prueba
print(f'Distribución de clases en entrenamiento: {np.bincount(y_train)}')
print(f'Distribución de clases en validación: {np.bincount(y_val)}')
print(f'Distribución de clases en prueba: {np.bincount(y_test)}')

Distribución de clases en entrenamiento: [61361  5938]
Distribución de clases en validación: [8767  848]
Distribución de clases en prueba: [17533  1696]


### **Entrenamiento del Modelo**

In [13]:
# Función para la definición y compilación del modelo
def model_set_up(units_1, activation, dropout, units_2, learning_rate):
  input = tf.keras.layers.Input(shape = (8,))
  int_dense_1 = tf.keras.layers.Dense(units = units_1)(input)
  int_activ_1 = tf.keras.layers.Activation(activation = activation)(int_dense_1)
  if dropout:
    int_drop = tf.keras.layers.Dropout(0.2)(int_activ_1)
    int_dense_2 = tf.keras.layers.Dense(units = units_2)(int_drop)
  else:
    int_dense_2 = tf.keras.layers.Dense(units = units_2)(int_activ_1)
  int_activ_2 = tf.keras.layers.Activation(activation = activation)(int_dense_2)
  output = tf.keras.layers.Dense(units = 1, activation = "softmax")(int_activ_2)
  model = tf.keras.models.Model(inputs = [input], outputs = [output])
  model.compile(loss = tf.keras.losses.categorical_crossentropy, optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate), metrics=["accuracy"])
  return model

In [14]:
# Diccionario con la configuración de hiperparámetros a explorar
setting = {"units_min": 64, "units_max": 512, "units_step": 32,
           "activation": ["relu", "sigmoid", "tanh"],
           "learning_rate_min": 1e-4, "learning_rate_max": 1e-2, "learning_rate_sampling": "log"}

In [15]:
# Función para la configuración y creación del modelo
def hypermodel(hyperparameter):
  units_1 = hyperparameter.Int("units_1", min_value = setting["units_min"], max_value = setting["units_max"], step = setting["units_step"])
  activation = hyperparameter.Choice("activation", setting["activation"])
  dropout = hyperparameter.Boolean("dropout")
  units_2 = hyperparameter.Int("units_2", min_value = setting["units_min"], max_value = setting["units_max"], step = setting["units_step"])
  learning_rate = hyperparameter.Float("learning_rate", min_value = setting["learning_rate_min"], max_value = setting["learning_rate_max"], sampling = setting["learning_rate_sampling"])
  model = model_set_up(units_1 = units_1, activation = activation, dropout = dropout, units_2 = units_2, learning_rate = learning_rate)
  return model
hypermodel(keras_tuner.HyperParameters())

<Functional name=functional, built=True>

In [16]:
# Exploración de hiperparámetros
tuner = keras_tuner.RandomSearch(hypermodel = hypermodel, objective = "val_accuracy", max_trials = 30, seed = 42, project_name = "keras_tuner")
tuner.search_space_summary()

Search space summary
Default search space size: 5
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [17]:
# Exploración de hiperparámetros
tuner.search(X_train, y_train, validation_data = (X_val, y_val), epochs = 10, batch_size = 100)

Trial 30 Complete [00h 00m 19s]
val_accuracy: 0.08819552510976791

Best val_accuracy So Far: 0.08819552510976791
Total elapsed time: 00h 10m 33s


In [18]:
# Mejores hiperparámetro
best_model = tuner.get_best_models(num_models = 1)
best_model = best_model[0]
best_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 352)                 │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 352)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 192)                 │          67,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 192)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             193 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 71,137 (277.88 KB)

 Trainable params: 71,137 (277.88 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Mejor metrica
tuner.results_summary(1)

Results summary
Results in ./keras_tuner
Showing 1 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
units_1: 352
activation: relu
dropout: False
units_2: 192
learning_rate: 0.005977728042983696
Score: 0.08819552510976791


**Universidad Nacional de Colombia** - *Facultad de Ingeniería*